<center><img src="../图片数据/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

## 导入相关包，创建相机实例

In [1]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg
import PID
camera = Camera.instance(width=720, height=720)

## 创建相关控制变量

In [2]:
global face_x, face_y, face_w, face_h
face_x = face_y = face_w = face_h = 0
global target_valuex
target_valuex = 2100
global target_valuey
target_valuey = 2048

## 创建PID控制实例

In [3]:
xservo_pid = PID.PositionalPID(1.9, 0.3, 0.35)
yservo_pid = PID.PositionalPID(1.5, 0.2, 0.3)

## 创建云台总线舵机实例

In [4]:
!echo 'yahboom' | sudo -S chmod 777 /dev/ttyTHS1

[sudo] password for jetbot: 

In [5]:

from servoserial import ServoSerial
servo_device = ServoSerial() 

serial Open!


## 创建显示控件

In [6]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
face_image = widgets.Image(format='jpeg', width=300, height=300)
face_image2 = widgets.Image(format='jpeg', width=300, height=300)
face_image3 = widgets.Image(format='jpeg', width=300, height=300)
display(face_image)
display(face_image2)

display(face_image3)



Image(value=b'', format='jpeg', height='300', width='300')

Image(value=b'', format='jpeg', height='300', width='300')

Image(value=b'', format='jpeg', height='300', width='300')

In [ ]:
import cv2
import numpy as np
#import rect

def transform(img, mp, dp):
    """
    透视变换，mp：原图像点， dp：目标图像点
    """
    global x
    global y
    # 修改点的格式
    pts1 = np.float32(mp)
    pts2 = np.float32(dp)
     

    M = cv2.getPerspectiveTransform(pts1, pts2)

    dst = cv2.warpPerspective(img, M, (x, y))
    return dst
 
 
def enhance(img):

    dst = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    th = cv2.adaptiveThreshold(dst, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 5)
    return th
 


while 1:
    frame = camera.value
    frame = cv2.resize(frame, (300, 300))
    
    image = frame.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    #使用canny算法进行边缘检测
    edged = cv2.Canny(blurred,0,50)
    orig_edged = edged.copy()
    contours, hierarchy = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)

    for c in contours:
        p = cv2.arcLength(c, True)   #计算封闭轮廓的周长或者曲线的长度
        approx = cv2.approxPolyDP(c, 0.02*p, True)  #指定0.02*p精度逼近多边形曲线，这种近似曲线为闭合曲线,因此参数closed为True

        if len(approx) == 4:    #如果逼近的是四边形
            target = approx     #则此轮廓为要找的轮廓
            break               #找到即跳出循环
    image_or = image.copy()
    #cv2.drawContours(image, [target], -1, (0, 255, 0), 2)
    cv2.polylines(image, [target], True, (0, 255, 0), 3)
    
    if target[0,0,0] > target[1,0,0]:
        target = target = target[[1,2,3,0]]
    
    x = 300
    y = 300 
    src = image.copy()
    src2 = image.copy()

    mp = [i[0] for i in target]
    dx = 8
    dp = [[dx, y-dx], [x-dx, y-dx], [x-dx, dx], [dx, dx]]
    docimg = transform(src, mp, dp)
    
    
    image_final = transform(frame, mp, dp)
    
    image_or = transform(src2, mp, dp)
    docimg2 = enhance(docimg)
    
    face_image.value = bgr8_to_jpeg(image)
    face_image2.value =  bgr8_to_jpeg(image_final)
    face_image3.value =  bgr8_to_jpeg(docimg2)


In [72]:

frame = camera.value
frame = cv2.resize(frame, (300, 300))

image = frame.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
#使用canny算法进行边缘检测
edged = cv2.Canny(blurred,0,50)
orig_edged = edged.copy()
contours, hierarchy = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
contours = sorted(contours, key = cv2.contourArea, reverse = True)

for c in contours:
    p = cv2.arcLength(c, True)   #计算封闭轮廓的周长或者曲线的长度
    approx = cv2.approxPolyDP(c, 0.02*p, True)  #指定0.02*p精度逼近多边形曲线，这种近似曲线为闭合曲线,因此参数closed为True

    if len(approx) == 4:    #如果逼近的是四边形
        target = approx     #则此轮廓为要找的轮廓
        break               #找到即跳出循环
image_or = image.copy()

In [73]:
target

array([[[ 96, 122]],

       [[ 50, 247]],

       [[141, 292]],

       [[147, 128]]], dtype=int32)

In [74]:
target = target[[2,3,0,1]]

In [75]:
#cv2.drawContours(image, [target], -1, (0, 255, 0), 2)
cv2.polylines(image, [target], True, (0, 255, 0), 3)


x = 300
y = 300 
src = image.copy()
src2 = image.copy()

mp = [i[0] for i in target]
dp = [[0, y], [x, y], [x, 0], [0, 0]]
docimg = transform(src, mp, dp)

image_or = transform(src2, mp, dp)
docimg2 = enhance(docimg)

face_image.value = bgr8_to_jpeg(image)
face_image2.value =  bgr8_to_jpeg(image_or)
face_image3.value =  bgr8_to_jpeg(docimg2)

In [76]:
display(face_image3)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [77]:
mp

[array([141, 292], dtype=int32),
 array([147, 128], dtype=int32),
 array([ 96, 122], dtype=int32),
 array([ 50, 247], dtype=int32)]

In [43]:
target[0,0]

array([ 81, 250], dtype=int32)

In [46]:
target

array([[[163, 159]],

       [[167, 216]],

       [[299, 296]],

       [[168, 217]]], dtype=int32)

In [47]:
target[0]

array([[163, 159]], dtype=int32)